### Import packages

In [ ]:
# Scientific computing
import numpy as np

# Visualisation
import matplotlib.pyplot as plt

# Import the storage tank Class
from cryoevap.storage_tanks import Tank

# Import Cryogen class
from cryoevap.cryogens import Cryogen

### Define all Variables

In [ ]:
# Cylindrical storage tank properties
Q_roof = 0 # Roof heat ingress / W
T_air = 293.15 # Temperature of the environment K

# Set overall heat transfer coefficient through the walls for liquid and vapour
U_L = 2e-2 # W/m2/K
U_V = 2e-2 # W/m2/K

# Specify wall heat partitioning fraction
eta_w= 0.7

# Specify tank operating pressure
P = 101325 # Pa

# Generate aspect ratio range
a = np.linspace(0.1,10,100)

### Variables of interest

# Vertically orientated cylindrical tank volume
V_tank = 165000 #m^3 | values of 80, 60000 and 165000

# Initial liquid filling / Dimensionless
LF = 0.80 # values of 0.95, 0.80 and 0.30

# Specify heat transfer rate at the bottom
# This will represent the heat conduction from the piping system
Q_b = 60000 # W, 0 for V 80, 1000 for V 60000 and 60000 for V 165000

q_b = 13.09


# Initialize cryogen
Compound = "hydrogen" # hydrogen, methane, ammonia, nitrogen

C_cryo = Cryogen(name = Compound)
C_cryo.set_coolprops(P)

### Generate data

In [ ]:
#Boil-off ratio list for plotting
BOR_plot = []

# Generate tanks in Aspect Ratio range
for Aspect_R in a:
    # Calcular diametro interno
    d_i = ((4 * V_tank)/(np.pi * Aspect_R))**(1/3) # internal diameter / m
    d_o = d_i * 1.02 # external diameter / m
    # Initialize tank
    mid_tank = Tank(d_i, d_o, V_tank, LF)

    Q_b=q_b*mid_tank.A_T
    
    mid_tank.set_HeatTransProps(U_L, U_V, T_air, Q_b, Q_roof, eta_w)

    mid_tank.U_roof = U_V
    
    # Set cryogen
    mid_tank.cryogen = C_cryo
    
    # Calculate BOR
    BOR = (mid_tank.b_l_dot * 24 * 3600) / (mid_tank.V * mid_tank.LF * mid_tank.cryogen.rho_L)
    
    # Save BOR value
    BOR_plot.append(BOR*100)
    #rint("BOR = %.3f %% | B_l = %.3f " % (BOR * 100, mid_tank.b_l_dot))
    #print(f"Aspect Ratio:{Aspect_R:.3f}, Altura: {mid_tank.l:.3f} , Diametro interno:{mid_tank.d_i:.3f}, Volumen:{mid_tank.V} \n")


### Show and save data plot

In [ ]:
# Create a plot
plt.plot(a, BOR_plot,"ro")

# Add labels and title
plt.xlabel('Razon de aspecto | a')
plt.ylabel('Boil-off Ratio | %')
plt.title(f'Tank Volume: {V_tank}, Liquid Filling: {LF*100}%, Compound: {Compound}')

# Display the plot

plt.savefig(f'{Compound}/V_{V_tank}_LF_{LF*100}_C_{Compound}.png')
plt.show()

### Optimization

### Parametros

In [ ]:
# Define vertical spacing
dz = 0.1

# Calculate number of nodes
n_z = 1 + int(np.round(mid_tank.l_V/dz, 0))

# Define dimensionless computational grid
mid_tank.z_grid = np.linspace(0, 1, n_z)

# Define evaporation time as the transient period
hours = 24
evap_time = 3600 * hours

# Time step to plot each vapour temperature profile
# mid_tank.plot_interval = evap_time/6

# Time step to record data, relevant for plotting integrated quantities such as
# the vapour to liquid heat transfer rate, Q_VL
mid_tank.time_interval = 60

# Show simulation duration and grid
print(f"Simulation time: {hours} h")
print(f"Grid length: {len(mid_tank.z_grid)}")

In [ ]:
from scipy.optimize import Bounds, minimize

# Minimum and maximum practical ranges
# of the aspect ratio
bounds = Bounds([0.1], [10])

# Define objective function
def BOR_function(AR):
    # Update aspect ratio
    # Calculate internal diameter
    d_i = ((4 * V_tank)/(np.pi * AR))**(1/3) # internal diameter / m
    d_o = d_i * 1.02 # external diameter / m
    
    # Initialize tank
    mid_tank = Tank(d_i, d_o, V_tank, LF)

    Q_b=q_b*mid_tank.A_T
    
    mid_tank.set_HeatTransProps(U_L, U_V, T_air, Q_b, Q_roof, eta_w)

    mid_tank.U_roof = U_V
    
    # Set cryogen
    mid_tank.cryogen = C_cryo

    # Execute simulation
    mid_tank.evaporate(evap_time)
    
    # Calculate BOR
    BOR = 1 - mid_tank.data['V_L'][-1] / mid_tank.data['V_L'][0]
    
    print("a = %.3f, BOR=%.3f" % (AR, BOR*100))
    
    return BOR  

# Initial liquid filling to optimise

x0 = 1 
res = minimize(BOR_function, x0, method='trust-constr', options={'verbose': 1}, bounds=bounds)

In [ ]:
from scipy.optimize import Bounds, minimize

# Minimum and maximum practical ranges
# of initial liquid filling

# 5% represent ballast voyage
# 95% is the safety limit for potential liquid thermal expansion
bounds = Bounds([0.05], [0.95])

# Define objective function
def BOR_function(LF):

    # Update liquid filling
    mid_tank.LF = LF

    # Execute simulation
    mid_tank.evaporate(evap_time)

    # Calculate objective function
    f = 1 - mid_tank.data['V_L'][-1] / mid_tank.data['V_L'][0]
    
    print("LF = %.3f, f=%.3e" % (LF, f))
    return f    

# Initial liquid filling to optimise

x0 = 0.9 
res = minimize(BOR_function, x0, method='trust-constr', options={'verbose': 1}, bounds=bounds)

In [ ]:
res